In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# Теорія

## Означення

Кластерний аналіз (англ. Data clustering) — задача розбиття заданої вибірки об'єктів (ситуацій) на підмножини, які називаються кластерами, так, щоб кожен кластер складався зі схожих об'єктів, а об'єкти різних кластерів істотно відрізнялися. Задача кластеризації належить до статистичної обробки, а також до широкого класу завдань некерованого навчання.

Кластериза́ція ме́тодом k-сере́дніх (англ. k-means clustering) - впорядкування множини об'єктів у порівняно однорідні групи.

Мета методу - розділити $n$ спостережень на $k$ кластерів, так щоб кожне спостереження належало до кластера з найближчим до нього середнім значенням. Метод базується на мінімізації суми квадратів відстаней між кожним спостереженням та центром його кластера, тобто функції:

\begin{equation*}
    J = \sum_{i=1}^{n} \sum_{j=1}^{k} (x_i - с_j)^2
\end{equation*}

де $k$ — число кластерів, $n$ — число спостережень, $x_i$ - $i$-те спостереження, $c_j$ - центри мас $j$-го кластера.

Фунцкція $J$ - це функція втрат, котру для кластеризації ще називають `distortion`.

## Алгоритм методу «Кластеризація за схемою k-середніх»:

Маємо масив спостережень (об'єктів), кожен з яких має певні значення за рядом ознак. Відповідно до цих значень об'єкт розташовується у багатовимірному просторі.

1. Дослідник визначає кількість кластерів, що необхідно утворити.
2. Випадковим чином обирається $n$ спостережень, які на цьому кроці вважаються центрами кластерів.
3. Кожне спостереження «приписується» до одного з $k$ кластерів — того, відстань до якого найкоротша.
4. Розраховується новий центр кожного кластера як елемент, ознаки якого розраховуються як середнє арифметичне ознак об'єктів, що входять у цей кластер.
5. Відбувається така кількість ітерацій (повторюються кроки 3-4), поки кластерні центри стануть стійкими (тобто при кожній ітерації в кожен кластер потрапляють одні й ті самі об'єкти), дисперсія всередині кластера буде мінімізована, а між кластерами — максимізована.

Вибір кількості кластерів робиться на основі дослідницької гіпотези. Якщо її немає, то рекомендують спочатку створити 2 кластери, далі 3, 4, 5, порівнюючи отримані результати.

## Метод "ліктя"

Метод ліктя передбачає багаторазове циклічне виконання алгоритму зі збільшенням кількості кластерів, а також подальшим відкладанням на графіку функції втрат (`distortion`).

Характерный график выглядит так:

![Alt text](image/kmean.png)

Графік "ліктя" показує залежність функції втрат $V$ від кількості кластерів $k$. Якщо на графіку можна виокремити точку згину (так званий "лікоть"), то це може свідчити про те, що відповідна кількість кластерів є оптимальною.

## Посилання

1. [Алгоритм кластеризации Ллойда (K-средних, K-means)](https://www.youtube.com/watch?v=8vCuR1AndH0)
2. [Модель кластеризации KMeans](https://www.youtube.com/watch?v=EHZJMz6zyFE&ab_channel=machinelearrrning)
3. [Документация skikit-learn. 2.3. Кластеризация](https://scikit-learn.ru/clustering/#)
4. [sklearn.cluster.KMeans](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html)
5. [Кластеризуем лучше, чем «метод локтя»](https://habr.com/ru/companies/jetinfosystems/articles/467745/)
6. [Кластерный анализ](https://www.dmitrymakarov.ru/intro/clustering-16/)

# Функції

## Функція для пошуку кластерів

In [ ]:
def find_optimal_clusters(features, max_clusters, random_state=0) -> list:
    """
    Find the optimal number of clusters using the 'Elbow Method' for K-Means clustering.

    Parameters:
        data (array-like): The input data for clustering.
        max_clusters (int): The maximum number of clusters to consider.

    Returns:
        distortions: list of distorsions

    This function calculates the distortion (inertia) for different numbers of clusters
    ranging from 1 to max_clusters and plots a 'Elbow Method' graph to help choose the
    optimal number of clusters for K-Means clustering. The point where the distortion
    starts to decrease at a slower rate often indicates the optimal number of clusters.
    """
    distortions = []
    for i in range(1, max_clusters + 1):
        kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=100, n_init='auto', random_state=random_state).fit(features)
        distortions.append(kmeans.inertia_)
    
    plt.plot(range(1, max_clusters + 1), distortions)
    plt.plot(range(1, max_clusters + 1), distortions)
    plt.xticks(range(1, max_clusters + 1), rotation=45)
    plt.xlabel(r'Кількість кластерів, $k$')
    plt.ylabel(r'Функція втрат (дисторсія)')
    plt.title('Графік ліктя')
    plt.grid()
    plt.show()
    
    return distortions

## Функція для виконання алгоритму кластеризації за методом K-Means

In [ ]:
def perform_kmeans_clustering(features, n_clusters=8, random_state=0):
    """
    Performs clustering using the K-Means method and returns cluster labels and centroid coordinates.

    Parameters:
    - features: The array of features to cluster.
    - n_clusters: Number of clusters (default is 8).
    - random_state: The seed for generating random numbers (default 0).

    Returns:
    - cluster_labels: An array of cluster labels.
    - centroids: The coordinates of the centroids of the clusters.
    """

    # Create a K-Means object
    kmeans = KMeans(n_clusters=n_clusters, init='k-means++', max_iter=100, n_init='auto', random_state=0)

    # Clustering
    kmeans.fit(features)

    # Get cluster labels
    cluster_labels = kmeans.labels_

    # Пget the coordinates of centroids
    centroids = kmeans.cluster_centers_

    return cluster_labels, centroids

# Двовимірний датасет `data_2d`

In [ ]:
data_2d = pd.read_csv("data/data_2d.csv", header=None)
data_2d



Цей набір даних складається з 200 рядків і 3 стовпців. Перший стовпчик - це клас, який може набувати лише двох значень: $0$ або $1$. Другий стовпець - це перша ознака, яка є дійсним числом. Третій стовпець - це друга ознака, яка також є дійсним числом. 


Побудуємо розподіл даних.

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 4))

# Ознака 1 - Графік "ящик із вусами"
ax1.boxplot(data_2d[1], vert=False, widths=0.7, patch_artist=True, boxprops=dict(facecolor='lightblue'), medianprops={'color': 'red'})
ax1.set_title('Ознака 1')
ax1.set_yticks([])
ax1.spines['left'].set_visible(False)
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)

# Ознака 2 - Графік "ящик із вусами"
ax2.boxplot(data_2d[2], vert=False, widths=0.7, patch_artist=True, boxprops=dict(facecolor='lightcoral'), medianprops={'color': 'blue'})
ax2.set_title('Ознака 2')
ax2.set_yticks([])
ax2.spines['left'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)

# Діаграма розсіювання
scatter = ax3.scatter(data_2d[1], data_2d[2], c=data_2d[0], s=30, cmap='RdYlBu', alpha=0.75)
ax3.set_title('Діаграма розсіювання')
ax3.set_xlabel('Ознака 1')
ax3.set_ylabel('Ознака 2')
ax3.legend(*scatter.legend_elements(), title="Класи", loc='upper right')

plt.tight_layout()

# Додамо кольорові фони для кожного графіка
ax1.set_facecolor('#f7f7f7')
ax2.set_facecolor('#f7f7f7')
ax3.set_facecolor('#f7f7f7')

plt.show()

In [ ]:
# Розрахунок кореляції
corr = np.corrcoef(data_2d[1], data_2d[2])
print(corr[0, 1].round(2))

На основі графіків та коефіцієнта кореляції можна зробити висновки про дані:
- Розподіл першої ознаки: нормальний.
- Розподіл другої ознаки: нормальний.
- Кореляція між ознаками: слабка.

## Знаходження оптимальної кількості кластерів

In [ ]:
features = data_2d.iloc[:, 1:]

Jk_data_2d = find_optimal_clusters(features, max_clusters=10)

Судячи з графіку число кластерів для `data_2d` дорівнює $2$. Крім того в наших даних перша колонка скоріше за все є класом ознаки. Таких ознак в даних також $2$-і.

## Запуск алгоритму класифікації

In [ ]:
k_data_2d = 2

In [ ]:
cluster_labels, centroids = perform_kmeans_clustering(features, n_clusters=k_data_2d)

X_ceterod = centroids[:, 0]
y_centroid = centroids[:, 1]

## Візуалізація результат роботи кластеризації

In [ ]:
# Кластери
plt.scatter(x=data_2d[1], y=data_2d[2], s=10, c=cluster_labels, cmap='rainbow')
# Центроіди
plt.scatter(X_ceterod, y_centroid, s=55, c=['blue', 'red'], marker='x')

# Класи з датасету
plt.scatter(data_2d[1], data_2d[2], c=data_2d[0], s=55, cmap='RdYlBu', alpha=0.2)

plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.title("Visualization of clusters using K-Means")
plt.show()

## Висновки

1. Метод "ліктя" дає число ознак що дорівнює $2$, що підтверджується самим датасетом, де перша колонка є класом, що якої відносяться ознаки. Таких ознак там теж $2$.
2. На графіку зображені кластери та центроїди. На кожну точку певного кластеру накладено "ореол" відомого нам класу с датасету. Бачимо, що алгоритм добре впорався і добре прокласифікував наші дані.

# Датасет `mnist`

In [ ]:
mnist = pd.read_csv("data/mnist.csv", header=None)
mnist

In [ ]:
X_mnist = mnist.iloc[:, 1:]
y_mnist = mnist.iloc[:, 0]

## Аналіз даних

In [ ]:
classes = mnist[0].unique()
classes.sort()
classes

Схоже, що датасет має $10$ кластерів. Ознаки мають характеризувати якусь цифру.
![Alt text](image/digit.png)

## Застосування PCA

PCA (Principal Component Analysis) - це метод, що використовується для зменшення розмірності даних і виділення найбільш інформативних ознак. Основна ідея PCA полягає в проєктуванні багатовимірних даних на меншу кількість вимірів (головних компонентів), при цьому максимізується збереження дисперсії даних. 

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_pca_mnist = pca.fit_transform(X_mnist)

In [ ]:
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'purple', 'orange', 'pink']

for i in range(10):
    plt.scatter(X_pca_mnist[y_mnist == i, 0], X_pca_mnist[y_mnist == i, 1], c=colors[i], label=f'Клас {i}')

plt.xlabel('Головна компонента 1')
plt.ylabel('Головна компонента 2')
plt.title('Візуалізація кластерів після PCA')
plt.legend(loc='best')
plt.show()

## Застосування методу "ліктя"

Схоже, що число кластерів дорівнює $3$. 

In [ ]:
from random import randint

Jk_mnist = find_optimal_clusters(X_mnist, max_clusters=20, random_state=randint(0, 4294967295))

In [ ]:
k_mnist = 3

In [ ]:
cluster_labels, centroids = perform_kmeans_clustering(X_pca_mnist, n_clusters=k_mnist)

In [ ]:
# Кластери
plt.scatter(x=X_pca_mnist[:, 0], y=X_pca_mnist[:, 1], s=10, c=cluster_labels, cmap='rainbow')
# Центроіди
plt.scatter(centroids[:, 0], centroids[:, 1], s=75, marker='x')

plt.show()

## Висновки

1. Ніякої кореляції з класами даними в датаскті `mnist` я не бачу.